In [1]:
import networkx as nx
import json
import gc

from networkx.classes.function import nodes
from pandas.core.indexes import base

get_attr = lambda line, attr : line.split("%s=\"" % attr, 1)[1].split('"', 1)[0]
id = 0
u = 0


def parse_switch(switch_dict, line):
    """Parses all the switch blocks.

    Parameters
    ----------
    lines : List[str]
        Read RR-graph file.

    Returns
    -------
    Dict[int, Dict[int, str]]
        A dictionary of dictionaries, indexed by block ids, block name and ptc nums,
        holding the pin names.
    """

    if line.startswith("<switch "):
        id = int(get_attr(line, "id"))
        name = get_attr(line, "name")
        type = get_attr(line, "type")
        switch_dict.update({id : {'name' : name, 'type' : type}})

def parse_block_pins(pin_dict, line):
    """Parses all the block pins.

    Parameters
    ----------
    lines : List[str]
        Read RR-graph file.

    Returns
    -------
    Dict[int, Dict[int, str]]
        A dictionary of dictionaries, indexed by block ids, block name and ptc nums,
        holding the pin names.
    """
    global bid
    if line.startswith("<block_type "):
        bid = int(get_attr(line, "id"))
        name = get_attr(line, "name")
        pin_dict.update({bid : {'name' : name}})

    if "<pin_class " in line:
        ptc = int(get_attr(line, "ptc"))
        name = line.split("</pin>")[0].split('>')[-1]
        pin_dict[bid].update({ptc : name})

def parse_segments(seg_dict, line):
    """Parses all segment declarations.

    Parameters
    ----------
    lines : List[str]
        Read RR-graph file.

    Returns
    -------
    Dict[int, str]
        A mapping from segment ids to segment names.
    """

    if line.startswith("<segment id"):
        seg_dict.update({int(get_attr(line, "id")) : get_attr(line, "name")})

def parse_grid(grid, line):
    """Parses the grid layout.

    Parameters
    ----------
    lines : List[str]
        Read RR-graph file.

    Returns
    -------
    Dict[Tuple[int], int]
        A dictionary holding block id for each tile.
    """

    if line.startswith("<grid_loc "):
        x = int(get_attr(line, 'x'))
        y = int(get_attr(line, 'y'))
        bid = int(get_attr(line, "block_type_id"))
        grid.update({'%d,%d'%(x, y) : bid})

def parse_edges(line, G):
    """Parses all RR-graph edges.

    Parameters
    ----------
    lines : List[str]
        Read RR-graph file.
    G : nx.DiGraph
        RR-graph with nodes added.

    Returns
    -------
    None
    """

#     if line.startswith("<edge "):
#         u = int(get_attr(line, "src_node"))
#         v = int(get_attr(line, "sink_node"))
#         switch = int(get_attr(line, "switch_id"))
#         if G.has_node(u) and G.has_node(v):
#             G.add_edge(u, v, switch_id = switch)
    if line.startswith("<edge "):
        u = int(get_attr(line, "src_node"))
        v = int(get_attr(line, "sink_node"))
        switch = int(get_attr(line, "switch_id"))   
        if G.has_node(u) and G.has_node(v):
            G.add_edge(u, v, switch_id = switch)
#             G.add_edge(u, v, src_xl = G.nodes[u]["xlow"], src_yl = G.nodes[u]["ylow"], src_xh = G.nodes[u]["xhigh"], src_yh = G.nodes[u]["yhigh"], src_seg = G.nodes[u]["seg"], 
#                       destin_xl = G.nodes[v]["xlow"], destin_yl = G.nodes[v]["ylow"], destin_xh = G.nodes[v]["xhigh"], destin_yh = G.nodes[v]["yhigh"],destin_seg = G.nodes[v]["seg"], switch_id = switch)
        

def edge_direction(type, direction):
    # 0-->Bi_Dir 1-->INC_DIR(?), SINK and IPIN -1-->DEC_DIR(?), SOURCE and OPIN
    if "CHANY" or "CHANX" in type:
        if direction == "BI_DIR":
            return 0
        # else:
        #     return 1 if direction == "INC_DIR" else -1

    if "SINK" or "IPIN" in type:
        return 1
    else:
        return -1


def find_edge(G, connection_graph:dict, switch_dict):
    """
    1.Extract ID from connection_graph
    2.Search all edge information of this ID, only as the source. Record Source, Sink, and Switch ID
    """
    edge_graph = {}
    history =[]
    for key in connection_graph:
        if key == "counter":
            break
        i = 1
        
        if key not in edge_graph:
            edge_graph.update({key:{'source&destin' : connection_graph[key]}})

        
        for predecessor in G.predecessors(key):
            
            if '(%d, %d)'%(predecessor, key) in history:
                continue
            else:
                history.append('(%d, %d)'%(predecessor, key))
                
            edge_graph[key].update({'switch_id_4source_%d' %(i) : G.edges[predecessor,key]['switch_id'], 
                            'switch_id_4source_%d' %(i) : switch_dict[G.edges[predecessor,key]['switch_id']]})
            edge_graph[key].update({'source_%d' %(i): G.nodes[predecessor]})
            
            i += 1
            
            txt = "<edge source_id=%s source_seg=%s source_coord=(%s, %s)-(%s, %s) destin_id=%s destin_seg=%s destin_coord=(%s, %s)-(%s, %s) switch_name=%s\n>"\
                %(predecessor, G.nodes[predecessor]["seg"],G.nodes[predecessor]["xlow"], G.nodes[predecessor]["ylow"], G.nodes[predecessor]["xhigh"], G.nodes[predecessor]["yhigh"],
                key, connection_graph[key]["seg"], connection_graph[key]["xlow"], connection_graph[key]["ylow"], connection_graph[key]["xhigh"], connection_graph[key]["yhigh"], switch_dict[G.edges[predecessor,key]['switch_id']])
            with open(r'/home/ymen/Desktop/node.txt', 'a') as f:
                print(txt, file=f)
                
        i = 1
        
        for successor in G.successors(key):
            
            if '(%d, %d)'%(key, successor) in history:
                continue
            else:
                history.append('(%d, %d)'%(key, successor))
                
            edge_graph[key].update({'switch_id_4destin_%d' %(i) : G.edges[key,successor]['switch_id'], 
                                    'switch_id_4destin_%d' %(i) : switch_dict[G.edges[key,successor]['switch_id']]})
            edge_graph[key].update({'destin_%d' %(i) : G.nodes[successor]})
            
            i += 1
            txt = "<edge source_id=%s source_seg=%s source_coord=(%s, %s)-(%s, %s) destin_id=%s destin_seg=%s destin_coord=(%s, %s)-(%s, %s) switch_name=%s\n>"\
                    %(key, connection_graph[key]["seg"], connection_graph[key]["xlow"], connection_graph[key]["ylow"], connection_graph[key]["xhigh"], connection_graph[key]["yhigh"],
                    successor, G.nodes[successor]["seg"],G.nodes[successor]["xlow"], G.nodes[successor]["ylow"], G.nodes[successor]["xhigh"], G.nodes[successor]["yhigh"],switch_dict[G.edges[key,successor]['switch_id']])
                
            with open(r'/home/ymen/Desktop/node.txt', 'a') as f:
                print(txt, file=f)

            
        
        
#     for key in connection_graph:
#         if key == 'counter': 
#             break

#         i = 1
        
#         if not key in edge_graph:
#             edge_graph.update({key:{}})
        
#         direct = edge_direction(connection_graph[key]["base_type"], connection_graph[key]["direction"]) 

#         #No need to find predecessors
#         if direct == 1:
#             edge_graph[key].update({'destin' : connection_graph[key]})

#             for predecessor in G.predecessors(key):
#                 if '(%d, %d)'%(predecessor, key) in history:
#                     continue
#                 else:
#                     history.append('(%d, %d)'%(predecessor, key))
#                     edge_graph[key].update({'switch_id_%d' %(i) : G.edges[predecessor,key]['switch_id'], 
#                                     'switch_name_%d' %(i) : switch_dict[G.edges[predecessor,key]['switch_id']]})
#                     edge_graph[key].update({'source_%d' %(i): G.nodes[predecessor]})
#                     i += 1

#             edge_graph[key].update({'counter' : "%d" %(i-1), 'direction' : "1"})
#             continue
        
#         if direct == -1:
#             edge_graph.update({key:{'source' : connection_graph[key]}})

#             for successor in G.successors(key):
#                 if '(%d, %d)'%(key, successor) in history:
#                     continue
#                 else:
#                     history.append('(%d, %d)'%(key, successor))
#                     edge_graph[key].update({'switch_id_%d' %(i) : G.edges[key,successor]['switch_id'], 
#                                     'switch_name_%d' %(i) : switch_dict[G.edges[key,successor]['switch_id']]})
#                     edge_graph[key].update({'destin_%d' %(i) : G.nodes[successor]})
#                     i += 1

#             edge_graph[key].update({'counter' : "%d" %(i-1), 'direction' : "-1"})
#             continue
        
#         if direct == 0:
#             edge_graph.update({key:{'source&destin' : connection_graph[key]}})
            
#             for predecessor in G.predecessors(key):
#                 if '(%d, %d)'%(predecessor, key) in history:
#                     continue
#                 else:
#                     history.append('(%d, %d)'%(predecessor, key))
#                     edge_graph[key].update({'switch_id_4source_%d' %(i) : G.edges[predecessor,key]['switch_id'], 
#                                     'switch_name_4source_%d' %(i) : switch_dict[G.edges[predecessor,key]['switch_id']]})
#                     edge_graph[key].update({'source_%d' %(i): G.nodes[predecessor]})
#                     i += 1
#                     txt = "<edge source_id=%s source_seg=%s source_coord=(%s, %s)-(%s, %s) destin_id=%s destin_seg=%s destin_coord=(%s, %s)-(%s, %s) switch_name=%s\n>"\
#                         %(predecessor, G.nodes[predecessor]["seg"],G.nodes[predecessor]["xlow"], G.nodes[predecessor]["ylow"], G.nodes[predecessor]["xhigh"], G.nodes[predecessor]["yhigh"],
#                         key, connection_graph[key]["seg"], connection_graph[key]["xlow"], connection_graph[key]["ylow"], connection_graph[key]["xhigh"], connection_graph[key]["yhigh"], switch_dict[G.edges[predecessor,key]['switch_id']])
#                     with open(r'/home/ymen/Desktop/node.txt', 'a') as f:
#                         print(txt, file=f)
#             edge_graph[key].update({'counter_4source' : "%d" %(i-1)})

#             i = 1

#             for successor in G.successors(key):
#                 if '(%d, %d)'%(key, successor) in history:
#                     continue
#                 else:
#                     history.append('(%d, %d)'%(key, successor))
#                 edge_graph[key].update({'switch_id_4destin_%d' %(i) : G.edges[key,successor]['switch_id'], 
#                                 'switch_name_4destin_%d' %(i) : switch_dict[G.edges[key,successor]['switch_id']]})
#                 edge_graph[key].update({'destin_%d' %(i) : G.nodes[successor]})
#                 i += 1
#                 txt = "<edge source_id=%s source_seg=%s source_coord=(%s, %s)-(%s, %s) destin_id=%s destin_seg=%s destin_coord=(%s, %s)-(%s, %s) switch_name=%s\n>"\
#                     %(key, connection_graph[key]["seg"], connection_graph[key]["xlow"], connection_graph[key]["ylow"], connection_graph[key]["xhigh"], connection_graph[key]["yhigh"],
#                     successor, G.nodes[successor]["seg"],G.nodes[successor]["xlow"], G.nodes[successor]["ylow"], G.nodes[successor]["xhigh"], G.nodes[successor]["yhigh"],switch_dict[G.edges[key,successor]['switch_id']])
                
#                 with open(r'/home/ymen/Desktop/node.txt', 'a') as f:
#                         print(txt, file=f)

#             edge_graph[key].update({'counter_4destin' : "%d" %(i-1), 'direction' : "0"})
                
    return edge_graph

def long_straight(x_target, y_target, xl_input, yl_input, xh_input, yh_input, tile_type):
    """
    NN6BEG&NN2BEG
    SS6BEG&SS2BEG
    WW6BEG&WW2BEG
    EE6BEG&EE2BEG
    """
    nn_dict = ["NN6BEG", "NN2BEG"]
    ee_dict = ["EE2BEG", "EE6BEG"]
    ww_dict = ["WW6BEG", "WW2BEG"]
    ss_dict = ["SS6BEG", "SS2BEG"]
    if xl_input == x_target and yl_input == y_target:
        if tile_type in nn_dict or tile_type in ee_dict:
            return 1
    if xh_input == x_target and yh_input == y_target:
        if tile_type in ss_dict or tile_type in ww_dict:
            return 1
    return 0

def short_straight():
    pass
def tilt_straight():
    pass

def get_connection_graph(G, x, y):
    """Get all connections of (x, y) coordinate.

    Parameters
    ----------
    G : nx.DiGraph
        Parsed RR-graph.
    x : int
        x-coordinate of the requested tile.
    y : int
        y-coordinate of the requested tile.

    Returns
    -------
    connection
    """
    nn_dict = ["NN6BEG", "NN4BEG", "NN2BEG"]
    ee_dict = ["EE2BEG", "EE4BEG", "EE6BEG"]
    ww_dict = ["WW6BEG", "WW4BEG", "WW2BEG"]
    ss_dict = ["SS6BEG", "SS4BEG", "SS2BEG"]
    
    index = 0
    connection = {}
    tmp = {'counter' : {}}
    for u, attrs in G.nodes(data = True):
        #Get all nodes starting, ending at this tile, or pin of this tile
#         if (attrs["xlow"] == x and attrs["ylow"] == y) or (attrs["xhigh"] == x and attrs["yhigh"] == y):
#             connection.update({u : {'base_type' : attrs["base_type"], 'direction' : attrs["direction"], 'ptc' : attrs["ptc"], 
#                 'xlow' : attrs["xlow"],'xhigh' : attrs["xhigh"],'ylow' : attrs["ylow"],'yhigh' : attrs["yhigh"],'seg' : attrs["seg"]}})    
    
        index = 0
    
        if attrs["xlow"] == x and attrs["ylow"] == y and (attrs["seg"] in nn_dict or attrs["seg"] in ee_dict):
            connection.update({u : {'base_type' : attrs["base_type"], 'direction' : attrs["direction"], 'ptc' : attrs["ptc"], 
                'xlow' : attrs["xlow"],'xhigh' : attrs["xhigh"],'ylow' : attrs["ylow"],'yhigh' : attrs["yhigh"],'seg' : attrs["seg"]}})                
            index = 1
            
        if attrs["xhigh"] == x and attrs["yhigh"] == y and (attrs["seg"] in ww_dict or attrs["seg"] in ss_dict):
            connection.update({u : {'base_type' : attrs["base_type"], 'direction' : attrs["direction"], 'ptc' : attrs["ptc"], 
                'xlow' : attrs["xlow"],'xhigh' : attrs["xhigh"],'ylow' : attrs["ylow"],'yhigh' : attrs["yhigh"],'seg' : attrs["seg"]}})                
            index = 1    
        
        if index :
            if attrs['seg'] != "None":
                if attrs["seg"] not in tmp['counter']:
                    tmp['counter'].update({attrs["seg"] : 1})
                else:
                    tmp['counter'][attrs["seg"]] += 1
            txt = "<node direction=%s id=%s type=%s coordinate=(%s, %s)-(%s, %s) segment_name=%s>\n"\
                    %(attrs["direction"], u, attrs["base_type"], attrs["xlow"], attrs["ylow"], attrs["xhigh"], attrs["yhigh"], attrs["seg"])
            with open(r'/home/ymen/Desktop/node.txt', 'a') as f:
                print(txt, file=f)
                    
    connection.update(tmp)

    return connection


def generate_graph_node(G, seg_dict, line): 
    global u
    if line.startswith("<node "):
        u = int(get_attr(line, "id"))
        base_type = get_attr(line, "type")
        ptc = int(get_attr(line, "ptc"))
        xlow = int(get_attr(line, "xlow"))
        xhigh = int(get_attr(line, "xhigh"))
        ylow = int(get_attr(line, "ylow"))
        yhigh = int(get_attr(line, "yhigh"))

        if base_type == "CHANY" or base_type == "CHANX":
            direction = get_attr(line, "direction")
        else:
            direction = '0'

        G.add_node(u, base_type = base_type, direction = direction, ptc = ptc, xlow = xlow, xhigh = xhigh, ylow = ylow, yhigh = yhigh, seg = "None")

    try:
        G.nodes[u]["seg"] = seg_dict[int(get_attr(line, "segment_id"))]
    except:
        pass

def crossing(x1, y1, x2, y2, x3, y3, x4, y4, target_x, target_y):
    if x1 == x2 and y3 == y4:
        return True if x1 == target_x and y3 == target_y else False
    if x3 == x4 and y1 == y2:
        return True if x3 == target_x and y1 == target_y else False

       

In [2]:
rr = r"/home/ymen/Desktop/rr_graph/test_full_size.xml"
with open(rr, "r") as inf:
    lines = inf.readlines()

In [3]:



switch_dict = {}
seg_dict = {}
pin_dict = {}
grid_dict = {}
edge = {}
G = nx.DiGraph()

with open(r'/home/ymen/Desktop/rr_graph/part_rr_graph.xml', 'a') as f:
    for line in lines:
        parse_switch(switch_dict, line)
        parse_segments(seg_dict, line)
        parse_block_pins(pin_dict, line)
        parse_grid(grid_dict, line)
        generate_graph_node(G, seg_dict, line)
        if line.startswith("<edge "):
            print(line, file=f)
del lines
del switch_dict
del pin_dict
del grid_dict
gc.collect()

# with open(r'/home/myz1237/Desktop/output.txt', 'w') as f:
#     print(json.dumps(switch_dict, indent=4), file=f)
#     print(json.dumps(seg_dict, indent=4), file=f)
#     print(json.dumps(pin_dict, indent=4), file=f)
#     print(json.dumps(grid_dict, indent=4), file=f)


0

In [4]:
with open(r'/home/ymen/Desktop/rr_graph/part_rr_graph.xml', 'r') as f:
    for line in f:
        parse_edges(line, G)
del f
gc.collect()

0

In [15]:
x1 = 40
y1 = 15

x2 = 45
y2 = 13

In [18]:
open(r'/home/ymen/Desktop/newnode_%d%d.txt'%(x2,y2), 'w').close()

target_coord = "(%d, %d)"%(x2, y2)
history = []

for u, v, attrs in G.edges(data=True):
    
    src_lcoord = "(%d, %d)"%(G.nodes[u]["xlow"], G.nodes[u]["ylow"])
    src_hcoord = "(%d, %d)"%(G.nodes[u]["xhigh"], G.nodes[u]["yhigh"])
    src_seg = G.nodes[u]["seg"]
    
    destin_lcoord = "(%d, %d)"%(G.nodes[v]["xlow"], G.nodes[v]["ylow"])
    destin_hcoord = "(%d, %d)"%(G.nodes[v]["xhigh"], G.nodes[v]["yhigh"])
    destin_seg = G.nodes[v]["seg"]
    

    
    txt = ""
    if attrs["switch_id"] == 5 and crossing(int(G.nodes[u]["xlow"]), int(G.nodes[u]["ylow"]), int(G.nodes[u]["xhigh"]), int(G.nodes[u]["yhigh"]),
                                           int(G.nodes[v]["xlow"]), int(G.nodes[v]["ylow"]), int(G.nodes[v]["xhigh"]), int(G.nodes[v]["yhigh"]), x2, y2):
        if "%d %d"%(u, v) in history:
            continue
        else:
            history.append("%d %d"%(u, v))
            history.append("%d %d"%(v, u))
            history.append("%d"%(u))
            history.append("%d"%(v))
            
        txt = "<short source_id=\"%s\" source_seg=\"%s\" source_coord=\"%s-%s\" destin_id=\"%s\" destin_seg=\"%s\" destin_coord=\"%s-%s\">"\
            %(u, G.nodes[u]["seg"], src_lcoord, src_hcoord, v, G.nodes[v]["seg"], destin_lcoord, destin_hcoord)
        with open(r'/home/ymen/Desktop/newnode_%d%d.txt'%(x2,y2), 'a') as f:
            print(txt, file=f)
        continue
        
for u, v, attrs in G.edges(data=True):
    
    src_lcoord = "(%d, %d)"%(G.nodes[u]["xlow"], G.nodes[u]["ylow"])
    src_hcoord = "(%d, %d)"%(G.nodes[u]["xhigh"], G.nodes[u]["yhigh"])
    src_seg = G.nodes[u]["seg"]
    
    destin_lcoord = "(%d, %d)"%(G.nodes[v]["xlow"], G.nodes[v]["ylow"])
    destin_hcoord = "(%d, %d)"%(G.nodes[v]["xhigh"], G.nodes[v]["yhigh"])
    destin_seg = G.nodes[v]["seg"]
        
    txt = ""
    
    if "%d"%(u) in history or "%d"%(v) in history:
        continue
        
    if src_lcoord == destin_lcoord and src_lcoord == target_coord:
        txt = "<edge source_id=\"%s\" source_seg=\"%s\" source_coord=\"%s-%s\" destin_id=\"%s\" destin_seg=\"%s\" destin_coord=\"%s-%s\">"\
            %(u, G.nodes[u]["seg"], src_lcoord, src_hcoord, v, G.nodes[v]["seg"], destin_lcoord, destin_hcoord)
        with open(r'/home/ymen/Desktop/newnode_%d%d.txt'%(x2,y2), 'a') as f:
            print(txt, file=f)
        continue
    if src_lcoord == destin_hcoord and src_lcoord == target_coord:
        txt = "<edge source_id=\"%s\" source_seg=\"%s\" source_coord=\"%s-%s\" destin_id=\"%s\" destin_seg=\"%s\" destin_coord=\"%s-%s\">"\
            %(u, G.nodes[u]["seg"], src_lcoord, src_hcoord, v, G.nodes[v]["seg"], destin_lcoord, destin_hcoord)
        with open(r'/home/ymen/Desktop/newnode_%d%d.txt'%(x2,y2), 'a') as f:
            print(txt, file=f)
        continue
    if src_hcoord == destin_lcoord and src_hcoord == target_coord:
        txt = "<edge source_id=\"%s\" source_seg=\"%s\" source_coord=\"%s-%s\" destin_id=\"%s\" destin_seg=\"%s\" destin_coord=\"%s-%s\">"\
            %(u, G.nodes[u]["seg"], src_lcoord, src_hcoord, v, G.nodes[v]["seg"], destin_lcoord, destin_hcoord)
        with open(r'/home/ymen/Desktop/newnode_%d%d.txt'%(x2,y2), 'a') as f:
            print(txt, file=f)
        continue
    if src_hcoord == destin_hcoord and src_hcoord == target_coord:
        txt = "<edge source_id=\"%s\" source_seg=\"%s\" source_coord=\"%s-%s\" destin_id=\"%s\" destin_seg=\"%s\" destin_coord=\"%s-%s\">"\
            %(u, G.nodes[u]["seg"], src_lcoord, src_hcoord, v, G.nodes[v]["seg"], destin_lcoord, destin_hcoord)
        with open(r'/home/ymen/Desktop/newnode_%d%d.txt'%(x2,y2), 'a') as f:
            print(txt, file=f)
        continue


In [17]:
sorted_seg_enum_source_column = ["SS6BEG", "WW4BEG", "NN6BEG", "EE4BEG", "SW6BEG", "NW6BEG", "NE6BEG", "SE6BEG", "SS2BEG", "WW2BEG", "NN2BEG", "EE2BEG", "SW2BEG", "NW2BEG", "NE2BEG", "EL1BEG", "ER1BEG", "SL1BEG", "SR1BEG", "WL1BEG", "WR1BEG", "NL1BEG", "NR1BEG"]
oblique_seg = ["SW6BEG", "NW6BEG", "NE6BEG", "SE6BEG","SW2BEG", "NW2BEG", "NE2BEG", "SE2BEG"]
sorted_seg_enum_source_column_special = ["SS6BEG", "WW4BEG", "NN6BEG", "EE4BEG", "SW6BEG", "NW6BEG", "NE6BEG", "SE6BEG", "SS2BEG", "WW2BEG", "NN2BEG", "EE2BEG", "SW2BEG", "NW2BEG", "SE2BEG", "NE2BEG", "EL1BEG", "ER1BEG", "SL1BEG", "SR1BEG", "WL1BEG", "WR1BEG", "NL1BEG", "NR1BEG",\
                                        "SR1BEG_S", "ER1BEG_S", "WR1BEG_S", "EL1BEG_N","WL1BEG_N", "NL1BEG_N"]
short_collect = []
counter = {}
with open(r'/home/ymen/Desktop/newnode_%d%d.txt'%(x2,y2), 'r') as f:
    counter.update({"crossing" : {}})
    for key in oblique_seg:
        counter["crossing"].update({"%s"%(key) : {}})
    
    for line in f:
        if line.startswith("<short"):
            source_id = get_attr(line, "source_id")
            source_seg = get_attr(line, "source_seg")
            source_coord = get_attr(line, "source_coord")
            destin_id = get_attr(line, "destin_id")
            destin_seg = get_attr(line, "destin_seg")
            destin_coord = get_attr(line, "destin_coord")
            txt = "%s %s"%(source_coord, destin_coord)
            counter["crossing"][source_seg].update({"%s&%s"%(source_id, destin_id) : txt})
with open(r'/home/ymen/Desktop/newnode_%d%d.txt'%(x2,y2), 'r') as f:
    for line in f:
        if line.startswith("<edge"):
            source_id = get_attr(line, "source_id")
            source_seg = get_attr(line, "source_seg")
            destin_id = get_attr(line, "destin_id")
            destin_seg = get_attr(line, "destin_seg")
            destin_coord = get_attr(line, "destin_coord")
            if destin_seg not in sorted_seg_enum_source_column_special:
                continue
            if destin_seg not in counter:
                counter.update({destin_seg : {}})
                counter[destin_seg].update({destin_id : destin_coord})
            else:
                counter[destin_seg].update({destin_id : destin_coord})
            if destin_seg in oblique_seg:
                i = 1
                for successor in G.successors(int(destin_id)):
                    if G.edges[int(destin_id), successor]["switch_id"] == 5:
                        txt = "(%s, %s)-(%s, %s)  %s"%(G.nodes[successor]["xlow"],G.nodes[successor]["ylow"], G.nodes[successor]["xhigh"], G.nodes[successor]["yhigh"], destin_coord)
                        counter[destin_seg].update({destin_id : txt})
             
for key in counter:
    if key == "crossing":
        for key1 in counter["crossing"]:
            counter["crossing"][key1].update({"counter" : len(counter["crossing"][key1])})
        continue
        
    counter[key].update({"counter" : len(counter[key])})
with open(r'/home/ymen/Desktop/extractnewnode_%d%d.txt'%(x2,y2), 'w') as f:
    print(json.dumps(counter, indent=4), file=f)
    
del counter
gc.collect()

46

In [7]:
open(r'/home/ymen/Desktop/node.txt', 'w').close()
connection_graph = get_connection_graph(G, x2, y2)
with open(r'/home/ymen/Desktop/output2.json', 'w') as f:
     print(json.dumps(connection_graph, indent=4), file=f)


In [8]:
edge = find_edge(G, connection_graph, switch_dict)
with open(r'/home/ymen/Desktop/edge2.json', 'w') as f:
     print(json.dumps(edge, indent=4), file=f)

NameError: name 'switch_dict' is not defined